In [ ]:
# notebook copied from here:
# https://github.com/urchade/GLiNER/blob/main/examples/finetune.ipynb

import json
from gliner import GLiNER
import spacy
from gliner_spacy.pipeline import GlinerSpacy
import torch
from tqdm import tqdm
from transformers import get_cosine_schedule_with_warmup
import os
import pandas as pd
from nervaluate import Evaluator

import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

import torch
from transformers import TrainerCallback
from gliner import GLiNERConfig, GLiNER
from gliner.training import Trainer, TrainingArguments
from gliner.data_processing.collator import DataCollatorWithPadding, DataCollator
from gliner.utils import load_config_as_namespace
from gliner.data_processing import WordsSplitter, GLiNERDataset

### Load NER Dataset for Fine-Tuning

**TODO**: you need to load your own NER dataset here

In [ ]:
# this function loads jsonl data
def read_jsonl(file_path):
    with open(file_path, "r") as f:
        data = [json.loads(line) for line in f]
    return data

In [ ]:
train_path = 'data/argilla_dataset_train.jsonl'
train_data_str_path = 'data/argilla_dataset_train_set_str.jsonl'
eval_path = 'data/argilla_dataset_eval.jsonl'
eval_data_str_path = 'data/argilla_dataset_eval_set_str.jsonl'
baseline_predictions_path = 'data/argilla_dataset_baseline_preds.jsonl'

train_data = read_jsonl(train_path)
train_data_str = read_jsonl(train_data_str_path)
eval_data = read_jsonl(eval_path)
eval_data_str = read_jsonl(eval_data_str_path)
baseline_predictions = read_jsonl(baseline_predictions_path)

len(train_data), len(train_data_str), len(eval_data), len(eval_data_str), len(baseline_predictions)

In [ ]:
# checking for gliner format
# {'tokenized_text' [], 'ner': [ [start_token_i, end_token_i, label], ...], ...}
print(f"{train_data[0]}\n\n{eval_data[3]}")

---

### Load Pre-Trained GLiNER

In [ ]:
# available models: https://huggingface.co/urchade

model = GLiNER.from_pretrained("urchade/gliner_largev2")

----

### Fine-Tuning

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps") # if you have apple m-series 
elif torch.cuda.is_available():
    device = torch.device("cuda") # if you have gpu
else:
    device = torch.device("cpu") # you most likely have this :)

model = GLiNER.from_pretrained("urchade/gliner_largev2")

# dynamic padding used to speed training up and save memory
data_collator = DataCollator(model.config, data_processor=model.data_processor, prepare_labels=True)

model.to(device)
print(f"Model is on: {next(model.parameters()).device}")

In [ ]:
num_epochs = 1  # Define how many times you want to pass over the dataset
batch_size = 16
data_size = len(train_data)
num_batches = data_size // batch_size  # Total batches per epoch
num_steps = num_epochs * num_batches  # Total training steps
print(f"Number of Epochs: {num_epochs}, Number of Batches: {num_batches}, Number of Steps: {num_steps}")

In [ ]:
training_args = TrainingArguments(
    output_dir="models",
    learning_rate=5e-6,
    weight_decay=0.01,
    others_lr=1e-5,
    others_weight_decay=0.01,
    lr_scheduler_type="linear", #cosine
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    focal_loss_alpha=0.75,
    focal_loss_gamma=2,
    num_train_epochs=num_epochs,
    evaluation_strategy="steps",
    #save_steps = 100,
    #save_total_limit=10,
    dataloader_num_workers = 0,
    use_cpu = False,
    #report_to="none",
    )

# this is to track loss during training
class LossTrackerCallback(TrainerCallback):
    def __init__(self):
        self.losses = []  # Store loss per step

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            self.losses.append(logs["loss"])
            print(f"Step {state.global_step}: Loss {logs['loss']}")

loss_tracker = LossTrackerCallback()

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    tokenizer=model.data_processor.transformer_tokenizer,
    data_collator=data_collator,
    callbacks=[loss_tracker]
)

trainer.train()

-----

### Save and Load Fine-tuned Model

In [ ]:
model.save_pretrained("models/fine-tuned-gliner")

In [ ]:
# possibly, remove from this cell onward from this section
fine_tuned_gliner = GLiNER.from_pretrained("models/fine-tuned-gliner", local_files_only=True)

In [ ]:
text = 'dress is a clotthing item. Look at that jacket yo!'

labels = ['clothing', 'organization', 'address', 'event']

entities = fine_tuned_gliner.predict_entities(text, labels)
print(entities)

In [ ]:
for example in train_data_str:
    entities = fine_tuned_gliner.predict_entities(example, labels)
    if entities:
        print(entities)

----

### Predict, Evaluate and Compare

In [ ]:
# let's load the fine-tuned model first

# fine-tuned model is the new GLiNER
fine_tuned_gliner = 'models/fine-tuned-gliner'

# use the same labels you have defined for this lab
labels = ['clothing', 'organization', 'address', 'event']

# config for fine-tuned gliner's spacy integration
custom_spacy_config = {
    "gliner_model": fine_tuned_gliner,
    "chunk_size": 250,
    "labels": labels,
    "style": "ent"
}

# Initialize a blank English spaCy pipeline and add fine-tuned GLiNER
nlp = spacy.blank("en")
nlp.add_pipe("gliner_spacy", config=custom_spacy_config)

text = "clothes clothing dress tshirt"

# Process the text with the pipeline
doc = nlp(text)

# Output detected entities
for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
# collect predictions of the fine-tuned model on the evaluation data
fine_tuned_preds = []
for example in eval_data_str:
    doc = nlp(example)
    doc_ents_nervaluate_format = []
    for ent in doc.ents:
        ent_nervulate_format = {'label': ent.label_, 'start': ent.start_char, 'end': ent.end_char}
        doc_ents_nervaluate_format.append(ent_nervulate_format)
    fine_tuned_preds.append(doc_ents_nervaluate_format)

In [ ]:
# below function converts the data to the format that nervaluate expects
def convert_data_to_nervaluate_format(data):
    formatted_data = []
    for data_point in data:
        formatted_data_point = [{'label': ner_point[2], 'start': ner_point[0], 'end': ner_point[1]} for ner_point in data_point['ner']]
        formatted_data.append(formatted_data_point)
    return formatted_data

In [ ]:
# convert the loaded data into nervaluate format
true = convert_data_to_nervaluate_format(eval_data)
baseline_preds = convert_data_to_nervaluate_format(baseline_predictions)

In [ ]:
# evaluate the baseline and get overall scores
baseline_evaluator = Evaluator(true, baseline_preds, tags=labels)
baseline_results, baseline_results_per_tag, baseline_result_indices, baseline_result_indices_by_tag = baseline_evaluator.evaluate()

print(f"Precision: {baseline_results['ent_type']['precision']}\nRecall: {baseline_results['ent_type']['recall']}\nF1: {baseline_results['ent_type']['f1']}")

print('----')

# evaluate the fine-tuned model and get overall scores
fine_tuned_evaluator = Evaluator(true, fine_tuned_preds, tags=labels)
fine_tuned_results, fine_tuned_results_per_tag, fine_tuned_result_indices, fine_tuned_result_indices_by_tag = fine_tuned_evaluator.evaluate()

print(f"Precision: {fine_tuned_results['ent_type']['precision']}\nRecall: {fine_tuned_results['ent_type']['recall']}\nF1: {fine_tuned_results['ent_type']['f1']}")

Let's observe the fine-tuned model's per-entity-type performance

In [ ]:
# this function returns the precision, recall, and f1 score for each entity type
# it's useful to understand how the model performs on each entity type
def get_per_entity_type_results(results_per_tag, labels):
    scores_per_entity_type = []
    for entity_type in labels:
        entity_type_scores = {'entity_type': entity_type,
                              'precision': results_per_tag[entity_type]['partial']['precision'],
                              'recall': results_per_tag[entity_type]['partial']['recall'],
                              'f1': results_per_tag[entity_type]['partial']['f1']
                              }
        scores_per_entity_type.append(entity_type_scores)

    return scores_per_entity_type

In [ ]:
baseline_per_entity_type_scores = get_per_entity_type_results(baseline_results_per_tag, labels)
fine_tuned_entity_type_scores = get_per_entity_type_results(fine_tuned_results_per_tag, labels)
baseline_scores = pd.DataFrame(baseline_per_entity_type_scores)
fine_tuned_scores = pd.DataFrame(fine_tuned_entity_type_scores)

In [ ]:
baseline_scores

In [ ]:
fine_tuned_scores